## Data transformation

This is the second automatically graded exercise for JODA. The objective here is to get our hands dirty with data. 

The context of this particular analysis is a fictional company that routinely runs different machine learning operations. 

We have generated a dataset that has the following columns or properties (to be engineered into features):

* Date
* Department
* ML Task ID
* ML Method
* Task Category
* Model Complexity (Parameters)
* Training Data Size (GB)
* Training Duration (Hours)
* Hardware Used
* Energy Consumption (kWh)
* CO2 Emissions (Kg)
* Cloud Provider

Moreover, there is a secondary dataset that includes information about the energy sources for the different cloud providers:

* Cloud Provider    
* Green Energy


TODO: Install the required packages using requirements.txt or individually

TODO: Import the needed packages   

In [246]:
import pandas as pd
import os 

TODO: Read the two data files. **Please note that the data files will be available under the data folder when running the grader.**

In [247]:
# Cloud providers
if os.path.isfile('data/cloud-providers.xlsx'):
    df_providers = pd.read_excel('data/cloud-providers.xlsx')
print(df_providers)

# CO2 emmissions
if os.path.isfile('data/co2-emissions.xlsx'):
    df_co2 = pd.read_excel('data/co2-emissions.xlsx')
df_co2.head()

  Cloud Provider Green Energy
0      CloudTech      Unknown
1         SkyNet      Unknown
2     DataNimbus      Unknown
3   EtherCompute        Green
4   VirtualStack       Hybrid


,Date,Department,ML Task ID,ML Method,Task Category,Model Complexity (Parameters),Training Data Size (GB),Training Duration (Hours),Hardware Used,Energy Consumption (kWh),CO2 Emissions (Kg),Cloud Provider
0,2024-02-01,Marketing,Task_1,RNN,Image Classification,2.008306e+06,44.885179,11.437543,CPU,150.145641,60.058256,EtherCompute
1,2024-02-01,Operations,Task_2,Linear Regression,Regression,1.053744e+03,12.288902,0.994852,TPU,12.260144,4.904058,CloudTech
2,2024-02-01,R&D,Task_3,Decision Tree,Machine Translation,1.098484e+03,127.561567,1.903623,CPU,27.869589,11.147836,CloudTech
3,2024-02-01,Customer Support,Task_4,Decision Tree,Text Generation,9.131620e+02,15.563152,2.348526,CPU,24.124306,9.649722,VirtualStack
4,2024-02-02,Finance,Task_5,Transformer,Clustering,8.023062e+06,4.043882,44.420819,TPU,559.037179,223.614871,DataNimbus


TODO: Join the two data frames to add information about the energy sources that the could providers use. [<code>merge()</code>](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) should be useful here.

 

In [248]:
df_joined = df_co2.merge(df_providers, how='left', on='Cloud Provider')
print(df_joined.head())

         Date        Department ML Task ID          ML Method  \
0  2024-02-01         Marketing     Task_1                RNN   
1  2024-02-01        Operations     Task_2  Linear Regression   
2  2024-02-01               R&D     Task_3      Decision Tree   
3  2024-02-01  Customer Support     Task_4      Decision Tree   
4  2024-02-02           Finance     Task_5        Transformer   

          Task Category  Model Complexity (Parameters)  \
0  Image Classification                   2.008306e+06   
1            Regression                   1.053744e+03   
2   Machine Translation                   1.098484e+03   
3       Text Generation                   9.131620e+02   
4            Clustering                   8.023062e+06   

   Training Data Size (GB)  Training Duration (Hours) Hardware Used  \
0                44.885179                  11.437543           CPU   
1                12.288902                   0.994852           TPU   
2               127.561567                   1.

TODO: Aggregate the data to department level. That is, each row should represents the aggregated values for each department. Notice that you do not need to aggregate each different column, only the ones instructed explicitly.

**Tip**: use the [<code>groubpy</code>](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html)-method in combination with [<code>agg</code>](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.agg.html).

TODO: Calculate the total of CO2 emissions for each department

In [249]:
df_co2_dep = df_joined.groupby('Department').agg({'CO2 Emissions (Kg)': 'sum'})
#df_co2_dep.reset_index(inplace=True)
print(df_co2_dep)

                  CO2 Emissions (Kg)
Department                          
Customer Support        12565.569898
Finance                 13568.637182
Human Resources         15256.236043
Marketing               12821.756125
Operations              15004.901708
R&D                     14644.874294


TODO: Rename CO2 emission column to <code>co2_emissions_kg</code>   

In [250]:
df_co2_dep.rename(columns={'CO2 Emissions (Kg)': 'co2_emissions_kg'}, inplace=True)
print(df_co2_dep)

                  co2_emissions_kg
Department                        
Customer Support      12565.569898
Finance               13568.637182
Human Resources       15256.236043
Marketing             12821.756125
Operations            15004.901708
R&D                   14644.874294


TODO: Create a function that picks the most common value among in a Pandas Series object

In [251]:
def pick_most_frequent(values):
    return values.mode()[0] if not values.mode().empty else None
#pick_most_frequent(pd.Series(['A', 'B', 'B', 'A']))

TODO: Pick the most frequent ML method for each department.

In [252]:
df_ml = df_joined.groupby('Department').agg({'ML Method': pick_most_frequent})
#df_ml.reset_index(inplace=True)
print(df_ml)

                          ML Method
Department                         
Customer Support  Linear Regression
Finance                         RNN
Human Resources                 RNN
Marketing             Decision Tree
Operations              Transformer
R&D                             RNN


TODO: Make sure that the rows are sorted according to CO2 emissions in a way that the department with the largest emissions is first.

In [253]:
df_co2_dep_sorted = df_co2_dep.sort_values(by='co2_emissions_kg', ascending=False)
print(df_co2_dep_sorted)

                  co2_emissions_kg
Department                        
Human Resources       15256.236043
Operations            15004.901708
R&D                   14644.874294
Finance               13568.637182
Marketing             12821.756125
Customer Support      12565.569898


TODO: Calculate the CO2 emissions for each department in different Green Energy categories. That is, the resulting dataframe will have as many colums as there are values for Green Energy.

Next, let's try to do something a bit more difficult. That is, calculate department CO2 emissions per energy type. 

One way to achieve this is to use pivot_table() function to create a separate dataframe with the new columns and join (using merge()) that to the main dataframe. We are sure there are even more clever ways. 

In [254]:
#df_providers['Green Energy'].value_counts()

df_co2_ge = df_joined.groupby(["Department", "Green Energy"]).agg({"CO2 Emissions (Kg)": "sum"}).reset_index()
df_co2_ge_pivot = df_co2_ge.pivot(index="Department", columns="Green Energy", values="CO2 Emissions (Kg)").fillna(0)
#df_co2_ge_pivot.reset_index(inplace=True)
print(df_co2_ge_pivot)

Green Energy            Green       Hybrid      Unknown
Department                                             
Customer Support  1463.697220  1425.139532  9676.733146
Finance           2991.539843  2427.085307  8150.012032
Human Resources   2423.439560  3431.874604  9400.921879
Marketing         2282.675323  2923.163567  7615.917236
Operations        2253.368372  3779.599277  8971.934059
R&D               3028.493500  2280.103568  9336.277226


TODO: Include the specified columns to the result dataframe, one per each energy type. 

In [244]:
df_co2_dep.reset_index(inplace=True)
print(df_co2_dep)
df_co2_dep_sorted.reset_index(inplace=True)
print(df_co2_dep_sorted)
df_co2_ge_pivot.reset_index(inplace=True)
print(df_co2_ge_pivot)

         Department  co2_emissions_kg
0  Customer Support      12565.569898
1           Finance      13568.637182
2   Human Resources      15256.236043
3         Marketing      12821.756125
4        Operations      15004.901708
5               R&D      14644.874294
         Department  co2_emissions_kg
0   Human Resources      15256.236043
1        Operations      15004.901708
2               R&D      14644.874294
3           Finance      13568.637182
4         Marketing      12821.756125
5  Customer Support      12565.569898
Green Energy        Department        Green       Hybrid      Unknown
0             Customer Support  1463.697220  1425.139532  9676.733146
1                      Finance  2991.539843  2427.085307  8150.012032
2              Human Resources  2423.439560  3431.874604  9400.921879
3                    Marketing  2282.675323  2923.163567  7615.917236
4                   Operations  2253.368372  3779.599277  8971.934059
5                          R&D  3028.493500  228

In [ ]:
df_final = df_co2_dep_sorted.merge(df_co2_ge_pivot, on="Department", how="outer")
df_final = df_final.merge(df_ml, on="Department", how="outer")
df_final = df_final.sort_values(by='co2_emissions_kg', ascending=False)

#print(df_co2_dep_sorted); print('\n')
#print(df_co2_ge_pivot); print('\n')
#print(df_ml); print('\n')
#print(df_final); print('\n')

                  co2_emissions_kg
Department                        
Human Resources       15256.236043
Operations            15004.901708
R&D                   14644.874294
Finance               13568.637182
Marketing             12821.756125
Customer Support      12565.569898


Green Energy            Green       Hybrid      Unknown
Department                                             
Customer Support  1463.697220  1425.139532  9676.733146
Finance           2991.539843  2427.085307  8150.012032
Human Resources   2423.439560  3431.874604  9400.921879
Marketing         2282.675323  2923.163567  7615.917236
Operations        2253.368372  3779.599277  8971.934059
R&D               3028.493500  2280.103568  9336.277226


                          ML Method
Department                         
Customer Support  Linear Regression
Finance                         RNN
Human Resources                 RNN
Marketing             Decision Tree
Operations              Transformer
R&D             

In [262]:
import os

def ensure_folder_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder '{folder_path}' created.")
    else:
        print(f"Folder '{folder_path}' already exists.")

ensure_folder_exists('results')

Folder 'results' already exists.


TODO Finally, save the results.

In [267]:
# df_final.to_excel('results/department_co2.xlsx', index=False)
df_final.to_pickle('results/department_co2.pkl')